In [57]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [58]:
#read elections data
dem = pd.read_csv('rawdata/dem_candidates.csv')
rep = pd.read_csv('rawdata/rep_candidates.csv')

## Naive Estimator

In [97]:
#binarize
dem['Won Primary 0_1'] = dem['Won Primary'].replace('No',0).replace('Yes',1) # outcome of election

rep['Won Primary 0_1'] = rep['Won Primary'].replace('No',0).replace('Yes',1) # outcome of election


### An "endorsed" candidate is one who has received at least one endorsement from the fields available in the dataset. 

For Democrats this meant any of the fields: 'Emily Endorsed?', 'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ', 'Sanders Endorsed?', 'Our Revolution Endorsed?',  'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',  'WFP Endorsed?', 'VoteVets Endorsed?' were yes.

For reps this meant any of the fields: 'Trump Endorsed?', 'Bannon Endorsed?', 'Great America Endorsed?', 'NRA Endorsed?', 'Right to Life Endorsed?', 'Susan B. Anthony Endorsed?', 'Club for Growth Endorsed?', 'Koch Support?', 'House Freedom Support?', 'Tea Party Endorsed?', 'Main Street Endorsed?', 'Chamber Endorsed?' were yes.

In [98]:
#split into treatment and control groups
#Treatment: endorsed
#Control: not endorsed

d_treatment = dem[(dem['Emily Endorsed?'] == 'Yes') | (dem['Guns Sense Candidate?'] == 'Yes') | (dem['Warren Endorsed? '] == 'Yes') | (dem['Sanders Endorsed?'] == 'Yes') | (dem['Our Revolution Endorsed?'] == 'Yes') | (dem['Justice Dems Endorsed?'] == 'Yes') | (dem['PCCC Endorsed?'] == 'Yes') | (dem['Indivisible Endorsed?'] == 'Yes') | (dem['WFP Endorsed?'] == 'Yes') | (dem['VoteVets Endorsed?'] == 'Yes')]
d_control = dem.loc[~dem.index.isin(d_treatment.index)]

r_treatment = rep[(rep['Trump Endorsed?'] == 'Yes') | (rep['Bannon Endorsed?'] == 'Yes') | (rep['Great America Endorsed?'] == 'Yes') | (rep['NRA Endorsed?'] == 'Yes') | (rep['Right to Life Endorsed?'] == 'Yes') | (rep['Susan B. Anthony Endorsed?'] == 'Yes') | (rep['Club for Growth Endorsed?'] == 'Yes') | (rep['Koch Support?'] == 'Yes') | (rep['House Freedom Support?'] == 'Yes') | (rep['Tea Party Endorsed?'] == 'Yes') | (rep['Main Street Endorsed?'] == 'Yes') | (rep['Chamber Endorsed?'] == 'Yes')]
r_control = rep.loc[~rep.index.isin(r_treatment.index)]


In [99]:
d_treatment_ones = pd.Series(np.zeros(len(dem))).rename('treatment')
d_treatment_ones.loc[d_treatment.index] = 1
dem['endorsed?'] = d_treatment_ones #treatment 1 = endorsed by any politician or group

r_treatment_ones = pd.Series(np.zeros(len(rep))).rename('treatment')
r_treatment_ones.loc[r_treatment.index] = 1
rep['endorsed?'] = r_treatment_ones #treatment 1 = endorsed by any politician or group

In [100]:
dem.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?,Primary Status 0_1,endorsed?,Won Primary 0_1
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0


### Difference In Means Estimate: 2018 Democrat Primary Candidates

In [101]:
np.mean(d_treatment['Won Primary 0_1'])

0.4485981308411215

In [102]:
np.mean(d_control['Won Primary 0_1'])

0.202991452991453

In [103]:
ATE = np.mean(d_treatment['Won Primary 0_1']) - np.mean(d_control['Won Primary 0_1'])
ATE

0.2456066778496685

In [104]:
Y = dem['Primary Status 0_1'].dropna()
Z = dem['endorsed?'].dropna()
Z = sm.add_constant(Z)

model = sm.OLS(Y,Z)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Primary Status 0_1   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     70.76
Date:                Sun, 09 May 2021   Prob (F-statistic):           1.81e-16
Time:                        17:57:30   Log-Likelihood:                -502.75
No. Observations:                 811   AIC:                             1010.
Df Residuals:                     809   BIC:                             1019.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2188      0.020     10.745      0.000       0.179       0.259
endorsed?      0.2719      0.032      8.412      0.000       0.208       0.335
==============================================================================
Omnibus:                      498.005   Durbin-Watson:                   2.190
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.362
Skew:                           0.657   Prob(JB):                     2.18e-23
Kurtosis:                       1.834   Cond. No.                         2.45
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### The ATE is 0.24, with p-value = 0, which indicates a significant positive correlation between dem endorsements and primary election results.

### Difference In Means Estimate: 2018 Republican Primary Candidates

In [105]:
np.mean(r_treatment['Won Primary 0_1'])

0.5379310344827586

In [106]:
np.mean(r_control['Won Primary 0_1'])

0.28298887122416533

In [107]:
ATE = np.mean(r_treatment['Won Primary 0_1']) - np.mean(r_control['Won Primary 0_1'])
ATE

0.2299983553533249

In [108]:
Y = rep['Primary Status 0_1'].dropna()
Z = rep['endorsed?'].dropna()
Z = sm.add_constant(Z)

model = sm.OLS(Y,Z)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Primary Status 0_1   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     36.13
Date:                Sun, 09 May 2021   Prob (F-statistic):           2.85e-09
Time:                        17:58:20   Log-Likelihood:                -496.97
No. Observations:                 774   AIC:                             997.9
Df Residuals:                     772   BIC:                             1007.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2830      0.018     15.414      0.000       0.247       0.319
endorsed?      0.2549      0.042      6.010      0.000       0.172       0.338
==============================================================================
Omnibus:                     1394.471   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              117.005
Skew:                           0.700   Prob(JB):                     3.92e-26
Kurtosis:                       1.708   Cond. No.                         2.67
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### The ATE is 0.23, with p-value = 0, which indicates a significant positive correlation between rep endorsements and primary election results.